In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import warnings
import torch
import numpy as np
import nltk
from nltk import sent_tokenize,word_tokenize

warnings.filterwarnings('ignore')

#downloading nltk data
try:
    nltk.download('punkt')
    nltk.download('stopwords')
    nltk.download('punkt_tab')
except:
    pass

#setting device
if torch.cuda.is_available():
    dev='cuda'
else:
    dev='cpu'

device=torch.device(device=dev)
print(f"The code is using device:{device}")

torch.manual_seed(42)
np.random.seed(42)

In [ ]:
# Data loading and pretraining

with open(file=r"/content/drive/MyDrive/wiki.train.tokens",mode="r",encoding='utf-8') as f:

    wiki_text=f.read()
    sentences=sent_tokenize(text=wiki_text)[:20000]

    pre_training_text=[]

    for i in range(0,len(sentences),3):
        pre_training_text.append(' '.join(sentences[i:i+3]))


print(pre_training_text)

In [ ]:
from sentence_transformers import SentenceTransformer
from transformers import GPT2Tokenizer

tokenizer=GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token=tokenizer.eos_token
print("Tokenizer initiated")

embedding_model=SentenceTransformer(model_name_or_path='sentence-transformers/all-MiniLM-L6-v2')
print("Embedding model initiated")

print(tokenizer)
print(embedding_model)

In [ ]:
file_path =r'/content/drive/MyDrive/extracted_text.txt'

with open(file_path, 'r', encoding='utf-8') as f:
    file_text = f.read()[:200000]
print("file document loaded successfully")

In [ ]:
# file_text

In [ ]:
# file_text='. '.join(file_text)

In [ ]:
# Process file document for fine-tuning
file_sentences = sent_tokenize(file_text)

finetune_data=[]

for sent in file_sentences:
    if len(sent)>20:
        finetune_data.append(sent.strip())

print(f"Processed {len(finetune_data)} fine-tuning samples from file document")

# Create document chunks for RAG retrieval
document_chunks = []
paragraphs = file_text.split('\n\n')
for para in paragraphs:
    para = para.strip()
    if len(para) > 50:  # Only meaningful paragraphs
        # Split long paragraphs into smaller chunks
        if len(para) > 300:
            sentences = sent_tokenize(para)
            for i in range(0, len(sentences), 2):
                chunk = ' '.join(sentences[i:i+2])
                if len(chunk) > 50:
                    document_chunks.append(chunk)
        else:
            document_chunks.append(para)

print(f"Created {len(document_chunks)} retrieval chunks")
print("Data loading complete!")

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import re
import os
from collections import Counter
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
!pip install torch_geometric

In [ ]:
# Instaling required graph libraries

import subprocess
import sys
from torch_geometric.nn import GCNConv, GATv2Conv, global_mean_pool
from torch_geometric.data import Data, Batch
print("PyTorch Geometric already installed")

In [ ]:
class CGTConfig:
    """Configuration for Contextual Graph Transformer"""

    def __init__(self):
        self.vocab_size = 50257
        self.hidden_dim = 384
        self.gnn_layers = 3
        self.transformer_layers = 4
        self.num_heads = 8
        self.gnn_type = 'gat'  # 'gcn' or 'gat'
        self.dropout = 0.1
        self.max_seq_len = 512

cgt_config=CGTConfig()

In [ ]:
class ContextualGraphTransformer(nn.Module):
    """Hybrid GNN + Transformer model for contextual understanding"""

    def __init__(self, config):
        super().__init__()
        self.config = config

        # Input token Embeddings
        self.embedding = nn.Embedding(config.vocab_size, config.hidden_dim)
        self.pos_embedding = nn.Embedding(config.max_seq_len, config.hidden_dim)

        # GNN Layers for capturing local context
        self.gnn_layers = nn.ModuleList()
        for i in range(config.gnn_layers):
            if config.gnn_type == 'gat':
                self.gnn_layers.append(GATv2Conv(config.hidden_dim, config.hidden_dim))
            else:
                self.gnn_layers.append(GCNConv(config.hidden_dim, config.hidden_dim))

        # Layer Normalization after GNN
        self.gnn_ln = nn.LayerNorm(config.hidden_dim)

        # Transformer Layers for capturing global context
        transformer_encoder_layer = nn.TransformerEncoderLayer(
            config.hidden_dim,
            nhead=8,
            dim_feedforward=4 * config.hidden_dim,
            dropout=config.dropout,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(transformer_encoder_layer, num_layers=config.transformer_layers)

        # Context Fusion Layer (Multi-Head Attention)
        self.fusion_attention = nn.MultiheadAttention(
            embed_dim=config.hidden_dim,
            num_heads=8,
            dropout=config.dropout,
            batch_first=True
        )

        # Output Layers
        self.ln_f = nn.LayerNorm(config.hidden_dim)
        # Enhanced output head: Two-layer FFN
        self.head_ffn = nn.Sequential(
            nn.Linear(config.hidden_dim, config.hidden_dim * 2),
            nn.ReLU(),
            nn.Dropout(config.dropout),
            nn.Linear(config.hidden_dim * 2, config.vocab_size)
        )

        # Calculate total parameters
        total_params = sum(p.numel() for p in self.parameters()) / 1e6
        print(f"CGT Model: {config.gnn_layers} GNN + {config.transformer_layers} Transformer layers outline ready")
        print(f"Total parameters: {total_params:.1f}M")

    def create_graph(self, input_ids):
        batch_size, seq_len = input_ids.shape
        graphs = []

        for b in range(batch_size):
            x = self.embedding(input_ids[b]) + self.pos_embedding(torch.arange(seq_len, device=input_ids.device))
            edge_index = []

            # Adjacent nodes
            for i in range(seq_len - 1):
                edge_index.append([i, i + 1])
                edge_index.append([i + 1, i])

            # Skip connections
            for i in range(seq_len - 2):
                edge_index.append([i, i + 2])
                edge_index.append([i + 2, i])

            if edge_index:
                edge_index = torch.tensor(edge_index, dtype=torch.long, device=input_ids.device).t().contiguous()
            else:
                edge_index = torch.empty((2, 0), device=input_ids.device, dtype=torch.long)

            graphs.append(Data(x, edge_index))

        return Batch.from_data_list(graphs)

    def forward(self, input_ids, attention_mask=None):
        graph_batch = self.create_graph(input_ids)
        x = graph_batch.x

        # GNN processing with dropout
        for gnn_layer in self.gnn_layers:
            x = F.dropout(x, p=self.config.dropout, training=self.training)
            x = F.relu(gnn_layer(x, graph_batch.edge_index))

        # Normalize GNN output
        x = self.gnn_ln(x)

        # Reshape for Transformer
        batch_size, seq_len = input_ids.shape
        x_gnn = x.view(batch_size, seq_len, -1)

        # Transformer processing
        if attention_mask is not None:
            attention_mask = attention_mask.bool()
            attention_mask = ~attention_mask  # Invert for transformer
        x_transformer = self.transformer(x_gnn, src_key_padding_mask=attention_mask)

        # Context fusion: Combine GNN and Transformer outputs
        x_fused, _ = self.fusion_attention(x_transformer, x_gnn, x_gnn)

        # Residual connection
        x = x_fused + x_transformer

        # Final processing
        x = self.ln_f(x)
        logits = self.head_ffn(x)

        return logits

model = ContextualGraphTransformer(config=cgt_config).to(device)
print(f"NEW CGT Model initialized with {sum(p.numel() for p in model.parameters())/1e6:.1f}M parameters")
print("Hybrid GNN + Transformer architecture ready!")
print(f"Architecture: {cgt_config.gnn_layers} GNN layers + {cgt_config.transformer_layers} Transformer layers")
print(f"Model type: {type(model).__name__}")

In [ ]:
len(finetune_data)

In [ ]:
def train_model(model,data,tokenizer,epochs=None,learning_rate=None,stage=None):
    """ Training function for both pretraining and fine-tuning"""

    # Optimizer and loss function
    optimizer=torch.optim.AdamW(model.parameters(),lr=learning_rate)
    loss_function=nn.CrossEntropyLoss()

    print(f"Starting {stage} for {epochs} epochs...")

    # Setting model in training mode
    model.train()

    for epoch in range(epochs):
        total_loss=0
        batch_count=0

        batch_size=16

        for i in range(0,len(data),batch_size):

            batched_data=data[i:i+batch_size]

            inputs=tokenizer(batched_data,
                      padding=True,
                      truncation=True,
                      max_length=128,
                      return_tensors='pt'
                      )

            input_ids=inputs['input_ids'].to(device)

            # Forward pass
            outputs = model(input_ids)

            # Calculate loss (next token prediction)
            shift_logits = outputs[..., :-1, :].contiguous()
            shift_labels = input_ids[..., 1:].contiguous()

            loss = loss_function(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            batch_count += 1

            if batch_count % 10 == 0:
                print(f"Epoch {epoch+1}/{epochs}, Batch {batch_count}, Loss: {loss.item():.4f}")

        avg_loss = total_loss / batch_count
        print(f"Epoch {epoch+1} completed, Average Loss: {avg_loss:.4f}")

    print(f" Training for {stage} completed successfully!")


In [ ]:
print(" Starting Pre_training phase")
train_model(model,pre_training_text,tokenizer,epochs=5,learning_rate=1e-5,stage="Pre_training")

In [ ]:
torch.save(model,r"/content/pre_training_model.pt")
print("Pre_trained model saved")

In [ ]:
print(" Starting Fine_tuning phase")
train_model(model,finetune_data,tokenizer,epochs=20,learning_rate=5e-5,stage="Fine_Tuning")

In [ ]:
torch.save(model.state_dict(), r"/content/fine_tuned_model.pt")
print("Fine-tuned model saved")

In [ ]:
class CGTBasedRAGSystem:

    def __init__(self,cgt_model, tokenizer, embedding_model, document_chunks):

        self.cgt_model=cgt_model
        self.tokenizer=tokenizer
        self.embedding_model=embedding_model
        self.document_chunks=document_chunks

        print("Computing embeddings for chunks")
        self.chunk_embeddings=self.embedding_model.encode(document_chunks)
        print(f"All embeddings for {len(document_chunks)} chunks are created Succesfully")


    def retrieve_relavent_chunks(self,query,top_k=None):

        query_embeddings=self.embedding_model.encode([query])

        similarity=cosine_similarity(query_embeddings,self.chunk_embeddings)

        top_indices = np.argsort(similarity[0])[-top_k:][::-1]

        relavent_chunks=[]

        for i in top_indices:
            relavent_chunks.append(self.document_chunks[i])

        return relavent_chunks,len(relavent_chunks)


    def clean_response(self, response):
        """Clean and format the response"""

        if not response:
            return ""

        # Remove extra whitespace
        response = ' '.join(response.split())

        # Ensure proper punctuation
        if response and not response.endswith(('.', '!', '?')):
            response += '.'

        return response


    def generate_with_cgt(self,query,relavent_context):

        related_chunks=' '.join(relavent_context)

        prompt = f"Context: {related_chunks[:500]}\n\nQuestion: {query}\nAnswer:"

        inputs=self.tokenizer(prompt,padding=True,truncation=True,max_length=200,return_tensors='pt')

        input_ids = inputs['input_ids'].to(device)
        attention_mask = inputs['attention_mask'].to(device)

        print(input_ids)
        print(attention_mask)

        # Generate using CGT model
        self.cgt_model.eval()
        with torch.no_grad():
            generated_tokens = []
            current_input = input_ids
            current_mask = attention_mask

            # Generate tokens one by one
            for step in range(50):  # Generate up to 50 tokens
                # Forward pass through CGT (GNN + Transformer)
                outputs = self.cgt_model(current_input, attention_mask=current_mask)

                # print(f"for step: {step} the outputs: {outputs}")

                # Get logits for the last token
                logits = outputs[:, -1, :]

                # Apply temperature for controlled generation
                temperature = 0.7
                logits = logits / temperature

                # Use top-k sampling
                top_k = 40
                top_k_logits, top_k_indices = torch.topk(logits, k=top_k)
                probabilities = F.softmax(top_k_logits, dim=-1)

                # Sample from top-k distribution
                next_token_idx = torch.multinomial(probabilities, 1)
                next_token_id = top_k_indices.gather(-1, next_token_idx)

                # Stop on EOS token
                if next_token_id.item() == self.tokenizer.eos_token_id:
                    break

                generated_tokens.append(next_token_id.item())

                # Update input for next iteration
                current_input = torch.cat([current_input, next_token_id], dim=-1)
                new_mask = torch.ones((1, 1), device=device)
                current_mask = torch.cat([current_mask, new_mask], dim=-1)

                # Stop at sentence boundaries for natural responses
                if len(generated_tokens) > 20:
                    decoded_so_far = self.tokenizer.decode(generated_tokens, skip_special_tokens=True)
                    if decoded_so_far.endswith('.') and step > 25:
                        break

            # Decode generated tokens
            if generated_tokens:
                generated_text = self.tokenizer.decode(generated_tokens, skip_special_tokens=True)
                generated_text = generated_text.strip()

                # Clean response
                if generated_text and len(generated_text.split()) >= 5:
                    return self.clean_response(generated_text)

        return self.intelligent_context_extraction(query, relavent_context)


    def intelligent_context_extraction(self, query, context_chunks):
        """Extract relevant information when model generation fails"""

        # Combine all context
        full_context = " ".join(context_chunks)

        # Split into sentences
        sentences = [s.strip() for s in full_context.split('.') if len(s.strip()) > 15]

        # Extract query keywords
        query_lower = query.lower()
        query_keywords = set([
            word.strip('.,!?()[]{}";:')
            for word in query_lower.split()
            if len(word) > 2 and word not in ['what', 'is', 'the', 'how', 'does', 'are', 'for', 'and', 'or', 'but']
        ])

        # Score sentences by relevance
        scored_sentences = []
        for sentence in sentences:
            sentence_lower = sentence.lower()
            sentence_words = set(word.strip('.,!?()[]{}";:') for word in sentence_lower.split())

            # Calculate relevance score
            keyword_overlap = len(query_keywords.intersection(sentence_words))
            if keyword_overlap > 0:
                scored_sentences.append((sentence, keyword_overlap))

        # Return best matching sentence
        if scored_sentences:
            scored_sentences.sort(key=lambda x: x[1], reverse=True)
            best_sentence = scored_sentences[0][0].strip()

            if best_sentence:
                # Ensure proper formatting
                best_sentence = best_sentence[0].upper() + best_sentence[1:]
                if not best_sentence.endswith(('.', '!', '?')):
                    best_sentence += '.'
                return best_sentence

        return "Information not found in the provided documentation."


    def question_answering(self,query):

        relavent_context,num_chunks=self.retrieve_relavent_chunks(query,top_k=3)

        answer=self.generate_with_cgt(query,relavent_context)

        formatted_response = f"""Question: {query}
        Answer: {answer}"""

        return formatted_response

# Initialize CGT-based RAG system (overriding previous)
print("Initializing CGT-based RAG system...")
cgt_rag_system = CGTBasedRAGSystem(model, tokenizer, embedding_model, document_chunks)
print("CGT-based RAG system ready!")
print("Using hybrid GNN + Transformer architecture")
print("Graph neural networks for local context")
print("Transformers for global dependencies")

In [ ]:
import time

print("TESTING CGT-BASED RAG SYSTEM (GNN + TRANSFORMER)")

test_questions = [
    "What Federated Learning does?"
]

print(f"Model Architecture: {type(model).__name__}")
print(f"Testing with {len(test_questions)} questions...")

for i, question in enumerate(test_questions, 1):

    start_time = time.time()
    response = cgt_rag_system.question_answering(question)
    generation_time = time.time() - start_time

    print(response)
    print(f"\nGeneration time: {generation_time:.2f} seconds")
    print(f"Architecture: GNN + Transformer hybrid processing")


print("CGT RAG SYSTEM TEST COMPLETE")